# Errors, or bugs, in your software

## Three classes of bugs

Let's discuss three major types of bugs in your code, from easiest to most difficult to diagnose:

1. **Syntax errors:** Errors where the code is not written in a valid way. (Generally easiest to fix.)
1. **Runtime errors:** Errors where code is syntactically valid, but fails to execute. Often throwing exceptions here. (Sometimes easy to fix, harder when in other's code.)
1. **Semantic errors:** Errors where code is syntactically valid, but contain errors in logic. (Can be difficult to fix.)

In [1]:
import numpy as np

### Syntax errors

In [2]:
print "This should only work in 2.X NOT used in this class."

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (4125897787.py, line 1)

In [3]:
print("This should only work in 3.x used in this class.)

SyntaxError: unterminated string literal (detected at line 1) (3701034456.py, line 1)

In [4]:
print("This should only work in 3.x used in this class.")

This should only work in 3.x used in this class.


### Runtime errors

In [5]:
# invalid operation
a = 0
5/a  # Division by zero

ZeroDivisionError: division by zero

In [6]:
# invalid operation
input = "40"
input/11  # Incompatiable types for the operation

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [7]:
while True:
    pass

KeyboardInterrupt: 

### Semantic errors

Say we're trying to confirm that a trigonometric identity holds. Let's use the basic relationship between sine and cosine, given by the Pythagorean identity

$$
\sin^2 \theta + \cos^2 \theta = 1
$$

We can write a function to check this:

In [8]:
import math

def pythagorean_identity(theta):
    return math.sin(theta)**2 + math.cos(theta)**4

In [9]:
pythagorean_identity(.4)

0.8713500597123389

## How to find and resolve bugs?

Debugging has the following steps:

1. **Detection** of an exception or invalid results. 
2. **Isolation** of where the program causes the error. This is often the most difficult step.
3. **Resolution** of how to change the code to eliminate the error. Mostly, it's not too bad, but sometimes this can cause major revisions in codes.


### Detection of Bugs

The detection of bugs is too often done by chance. While running your Python code, you encounter unexpected functionality, exceptions, or syntax errors. While we'll focus on this in today's lecture, you should never leave this up to chance in the future.

Software testing practices allow for thoughtful detection of bugs in software. We'll discuss more in the lecture on testing.

### Isolation of Bugs

There are three main methods commonly used for bug isolation:
1. The "thought" method. Think about how your code is structured and so what part of your could would most likely lead to the exception or invalid result.
2. Inserting ``print`` statements (or other logging techniques)
3. Using a line-by-line debugger like ``pdb``.

Typically, all three are used in combination, often repeatedly.

### Using `print` statements

Say we're trying to compute the **entropy** of a set of probabilities.  The
form of the equation is

$$
H = -\sum_i p_i \log(p_i)
$$

The choice of base for the logarithm varies for different applications. Base 2 gives the unit of bits, while base `e` gives nats, and base 10 gives units of "dits". 

We can write the function like this:

In [10]:
def entropy(p):
    items = []
    for p_i in p:
        interm = p_i * np.log2(p_i)
        items.append(interm)
    return -np.sum(items)

Imagine if we have the nucleotides in DNA: A, T, C & G for adenine, thymine, cytosine, and guanine, respectively.  If we have a given DNA sequence, e.g. `ATCGATCG` we can compute the observed probability for each nucleotide in the sequence as:

| Nucleotide | Occurrences | P(Nucleotide) |
|------------|-------------|---------------|
| A | 2 | .25 |
| T | 2 | .25 |
| C | 2 | .25 |
| G | 2 | .25 |

Thus, we have 4 states that are all equally likely, we can compute the entropy with the following:

In [11]:
entropy([.25, .25, .25, .25])

2.0

To reinforce this concept, let's imagine we want to encode the nucleotides A, T, C, G as bits.  Without any knowledge of nucleotide frequency, i.e. assuming they are all equally likely, we could define a bitwise representation of the nucleotides as:

| Nucleotide | Bit Representation |
|------------|--------------------|
| A | `00` |
| T | `01` |
| C | `10` |
| G | `11` |

We can see that it takes two bits to represent the nucleotides uniquely.  This is exactly what the `entropy` function predicts for the representation given they are all equality, e.g. `entropy([.25, .25, .25, .25])`

----

But there are some problems with this function.  Namely, nothing prevents the user from giving it the following inputs:

In [12]:
entropy([.5, .5, .5, .5])

2.0

Recall that the inputs to the function `entropy` are supposed to be probabilities.  The sum of probabilities of all states cannot exceed 1.  Why does the function let the user pass in probabilities that don't sum to 1?

Let's start by adding in some print statements to help us look at what is happening.

In [13]:
def entropy(p):
    print(f"p: {p}")
    items = []
    for p_i in p:
        print(f"p_i: {p_i}")
        interm = p_i * np.log2(p_i)
        print(f"interm: {interm}")
        items.append(interm)
    print(f"items: {items}")
    return -np.sum(items)

In [14]:
entropy([0.5, 1.2])

p: [0.5, 1.2]
p_i: 0.5
interm: -0.5
p_i: 1.2
interm: 0.31564128700055255
items: [-0.5, 0.31564128700055255]


0.18435871299944745

In [15]:
entropy([0.1, 0.7, 0.2])

p: [0.1, 0.7, 0.2]
p_i: 0.1
interm: -0.33219280948873625
p_i: 0.7
interm: -0.3602012209808308
p_i: 0.2
interm: -0.46438561897747244
items: [-0.33219280948873625, -0.3602012209808308, -0.46438561897747244]


1.1567796494470395

In [16]:
def entropy(p):
    """
     arg p: list of float
    """
    # begin by checking all inputs are between 0 and 1 inclusive
    check1 = []
    for ele in p:
        check1.append((ele <= 1) and (ele >= 0))
    else:
        pass
    # verify the sum of the probabilities is 1
    # note, the use of np.isclose is correct but the following 
    # may not return the expected result
    # check2 = 1 == np.sum(p)
    check2 = np.isclose(1, np.sum(p), atol=1e-08)
    if all(check1) and check2:
        items = p * np.log2(p)
        return -np.sum(items)
    else:
        return -1

In [17]:
all([True, True, True])

True

In [18]:
all([False, True, True])

False

In [19]:
np.isclose(1, .9999)

False

In [20]:
np.isclose(1, .999999)

True

In [21]:
np.isclose(1, .9999, atol=1e-03)

True

In [22]:
print(entropy([0.5, 0.5]))
print(entropy([0.25, 0.25, 0.25, 0.25]))

1.0
2.0


In [23]:
entropy([1.2, 0, .5])

-1

If we can't easily see the bug here, let's add print statements to see the variables change over time.

## NOTE:

1. Add print statements in tiered way, starting with simple print statements.
1. Point out that may need slight refactor on result.

    ```python
    def entropy(p):
        print(p)
        items = p * np.log(p)
        print(items)
        result = -np.sum(items)
        print(result)
        return result
    ```

1. Show complication of reading multiple print statements without labels.
1. Add labels so code looks like below.

    ```python
    def entropy(p):
        print("p=%s" % p)
        items = p * np.log(p)
        print("items=%s" % items)
        result = -np.sum(items)
        print("result=%s" % result)
        return result
    ```

Now it works fine for the first set of inputs. Let's try other inputs.

In [24]:
entropy([0.5, 0, .5])

/tmp/ipykernel_443/1281330399.py:17: RuntimeWarning: divide by zero encountered in log2
  items = p * np.log2(p)
/tmp/ipykernel_443/1281330399.py:17: RuntimeWarning: invalid value encountered in multiply
  items = p * np.log2(p)


nan

We should have documented the inputs to the function!

In [25]:
# Create a vector of probabilities.
p = np.arange(start=5., stop=-1., step=-0.5)
p /= np.sum(p)
p

array([ 0.18518519,  0.16666667,  0.14814815,  0.12962963,  0.11111111,
        0.09259259,  0.07407407,  0.05555556,  0.03703704,  0.01851852,
        0.        , -0.01851852])

In [26]:
entropy(p)

-1

We get ``nan``, which stands for "Not a Number". What's going on here?

Let's add our print statements again, but it only fails later in the range of numbers. We may choose to print only if we find a `nan`.

In [27]:
def entropy1(p):
    print("p=%s" % str(p))
    items = p * np.log2(p)
    if [np.isnan(el) for el in items]:
        print(items)
    else:
        pass
    return -np.sum(items)

In [28]:
entropy1([.1, .2])

p=[0.1, 0.2]
[-0.33219281 -0.46438562]


0.7965784284662087

In [29]:
entropy1(p)

p=[ 0.18518519  0.16666667  0.14814815  0.12962963  0.11111111  0.09259259
  0.07407407  0.05555556  0.03703704  0.01851852  0.         -0.01851852]
[-0.45054804 -0.43082708 -0.40813148 -0.38208756 -0.35221389 -0.31786661
 -0.27813981 -0.2316625  -0.17610694 -0.10657199         nan         nan]


/tmp/ipykernel_443/3489211204.py:3: RuntimeWarning: divide by zero encountered in log2
  items = p * np.log2(p)
/tmp/ipykernel_443/3489211204.py:3: RuntimeWarning: invalid value encountered in log2
  items = p * np.log2(p)
/tmp/ipykernel_443/3489211204.py:3: RuntimeWarning: invalid value encountered in multiply
  items = p * np.log2(p)


nan

By printing some of the intermediate items, we see the problem: 0 * np.log(0) is resulting in a NaN. Though mathematically it's true that limx→0[xlog(x)]=0limx→0[xlog⁡(x)]=0, the fact that we're performing the computation numerically means that we don't obtain this result.

Often, inserting a few print statements can be enough to figure out what's going on.


In [30]:
def entropy2(p):
    p = np.asarray(p)  # convert p to array if necessary
    print(p)
    items = []
    for val in p:
        item = val * np.log2(val)
        if np.isnan(item):
            print("%f makes a nan" % val)
        else:
            pass
        items.append(item)
    #items = p * np.log(ps)
    return -np.sum(items)

In [31]:
entropy2(p)

[ 0.18518519  0.16666667  0.14814815  0.12962963  0.11111111  0.09259259
  0.07407407  0.05555556  0.03703704  0.01851852  0.         -0.01851852]
0.000000 makes a nan
-0.018519 makes a nan


/tmp/ipykernel_443/2896936389.py:6: RuntimeWarning: divide by zero encountered in log2
  item = val * np.log2(val)
/tmp/ipykernel_443/2896936389.py:6: RuntimeWarning: invalid value encountered in double_scalars
  item = val * np.log2(val)
/tmp/ipykernel_443/2896936389.py:6: RuntimeWarning: invalid value encountered in log2
  item = val * np.log2(val)


nan

### Using Python's debugger, `pdb`

Python comes with a built-in debugger called [pdb](http://docs.python.org/2/library/pdb.html).  It allows you to step line-by-line through a computation and examine what's happening at each step.  Note that this should probably be your last resort in tracing down a bug.  I've probably used it a dozen times or so in five years of coding.  But it can be a useful tool to have in your toolbelt.

You can use the debugger by inserting the line
``` python
import pdb; pdb.set_trace()
```
within your script. To leave the debugger, type "exit()". To see the commands you can use, type "help".

Let's try this out:

In [42]:
def entropy(p):
    items = p * np.log2(p)
    import pdb; pdb.set_trace()
    return -np.sum(items)

This can be a more convenient way to debug programs and step through the actual execution.

In [43]:
p = [.1, -.2, .3]
entropy(p)

/tmp/ipykernel_443/3188088253.py:2: RuntimeWarning: invalid value encountered in log2
  items = p * np.log2(p)


> /tmp/ipykernel_443/3188088253.py(4)entropy()
      1 def entropy(p):
      2     items = p * np.log2(p)
      3     import pdb; pdb.set_trace()
----> 4     return -np.sum(items)

ipdb> a
p = [0.1, -0.2, 0.3]
ipdb> c


nan

In [44]:
p = "[0.1, 0.3, 0.5, 0.7, 0.9]"
entropy(p)

TypeError: ufunc 'log2' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''